In [1]:
from datetime import datetime, timedelta
from pprint import pprint

import japanize_matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from stockstats import StockDataFrame

sns.set(font="IPAexGothic", rc={"figure.figsize": (11, 8)})
pd.options.display.float_format = "{:6.2f}".format

/usr/local/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
def get_yf_data(tickers: str, start_str: datetime, end_str: datetime):
    return yf.download(
        tickers=tickers,
        start=start_str,
        end=end_str,
        interval="1d",
        group_by="ticker",
    ).dropna()

In [36]:
def convert_df_to_stock_df(df: pd.DataFrame) -> StockDataFrame:
    sdf = df.copy()
    sdf.rename(
        columns={
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Adj Close": "amount",
            "Volume": "volume",
        },
        inplace=True,
    )
    sdf.index.names = ["date"]
    return StockDataFrame(sdf)


def MACD(arr: pd.DataFrame) -> tuple[pd.Series, pd.Series]:
    sdf = convert_df_to_stock_df(arr)
    StockDataFrame.MACD_EMA_SHORT = 12
    StockDataFrame.MACD_EMA_LONG = 26
    StockDataFrame.MACD_EMA_SIGNAL = 9
    return (sdf["macd"], sdf["macds"])


def RSI(arr: pd.DataFrame, rsi: int) -> pd.Series:
    sdf = convert_df_to_stock_df(arr)
    return sdf["rsi_" + str(rsi)]


def SMA(arr: pd.DataFrame, payload=200) -> pd.Series:
    sdf = convert_df_to_stock_df(arr)
    return sdf["open_" + str(payload) + "_sma"]


def BB(arr: pd.DataFrame, payload=200, std_times=2) -> pd.Series:
    stock = convert_df_to_stock_df(arr)
    stock.BOLL_PERIOD = payload
    stock.BOLL_STD_TIMES = std_times
    return (stock["boll"], stock["boll_ub"], stock["boll_lb"])

In [49]:
class My_Strategy(Strategy):
    bb_payload = 150
    bb_std_times = 0.5
    stop_loss = 10  # %
    take_profit = 10  # %

    def init(self):
        self.macd, self.macd_signal = self.I(MACD, self.data.df)
        self.bb, self.bb_ub, self.bb_lb = self.I(
            BB, self.data.df, self.bb_payload, self.bb_std_times
        )

    def golden_cross_with_macd_day(self):
        return crossover(self.macd, self.macd_signal)

    def dead_cross_with_macd_day(self):
        return crossover(self.macd_signal, self.macd)

    def is_macd_over_zero(self):
        return self.macd[-1] > 0 and self.macd_signal[-1] > 0

    def is_long_potitison_by_sma(self):
        return self.bb[-1] < self.data.Close[-1]  # BBにSMAが入ってる

    def is_short_postition_by_sma(self):
        return self.bb[-1] > self.data.Close[-1]  # BBにSMAが入ってる

    def not_trade_by_bb(self):
        return (
            self.bb_ub[-1] >= self.data.Close[-1]
            and self.data.Close[-1] >= self.bb_lb[-1]
        )

    def next(self):
        # MACDがゴールデンクロスしたら、今までの注文を終了して買い注文
        if self.golden_cross_with_macd_day():
            if self.position.is_short:
                self.position.close()

            if (
                self.is_long_potitison_by_sma()
                and not self.is_macd_over_zero()
                and not self.not_trade_by_bb()
            ):
                self.buy(
                    # sl=self.data.Close[-1] * (1 - (self.stop_loss / 100)),
                    # tp=self.data.Close[-1] * (1 + (self.take_profit / 100)),
                )

        # MACDがデッドクロスしたら、今までの注文を終了して売り注文
        if self.dead_cross_with_macd_day():
            if self.position.is_long:
                self.position.close()

            if (
                self.is_short_postition_by_sma()
                and self.is_macd_over_zero()
                and not self.not_trade_by_bb()
            ):
                self.sell(
                    # sl=self.data.Close[-1] * (1 + (self.stop_loss / 100)),
                    # tp=self.data.Close[-1] * (1 - (self.take_profit / 100)),
                )

In [66]:
start_at = datetime.strptime("2000/01/01 00:00:00", "%Y/%m/%d %H:%M:%S")
end_at = datetime.strptime("2023/12/31 00:00:00", "%Y/%m/%d %H:%M:%S")

# NDX BEST SQN 2.15 My_Strategy(bb_payload=160,bb_std_times=0.7)
# SPY BEST SQN 1.71 My_Strategy(bb_payload=170,bb_std_times=0.8)
# BTC-USD BEST SQN 2.10 My_Strategy(bb_payload=150,bb_std_times=0.0)
# GC=F(GOLD) BEST SQN 0.78 My_Strategy(bb_payload=100,bb_std_times=0.9)
# JPY=X BEST 0.5 SQN My_Strategy(bb_payload=100,bb_std_times=0.9)
bt = Backtest(
    get_yf_data("JPY=X", start_at, end_at),
    My_Strategy,
    cash=10000000,
    commission=0.003,
    exclusive_orders=True,
)

# # 出力
# output = bt.run()
# print(output)
# bt.plot()

# # 最適化
optimize = bt.optimize(
    bb_payload=range(100, 200, 10),
    bb_std_times=list(map(lambda x: x / 10, range(0, 10, 1))),
    maximize="SQN",
)
bt.plot()
print(optimize)
print(optimize._strategy)


# # 最適化したものが通用するのか、違う期間で検証する。
# bt = Backtest(
#     get_yf_data("NDX", end_at, end_at + (end_at - start_at)),
#     My_Strategy,
#     cash=10000000,
#     commission=0.003,
#     exclusive_orders=True,
# )

# optimize = bt.optimize(
#     prop_rsi_buy=optimize._strategy.prop_rsi_buy,
#     prop_rsi_sell=optimize._strategy.prop_rsi_sell,
#     stop_loss=optimize._strategy.stop_loss,
#     take_profit=optimize._strategy.take_profit,
# )
# bt.plot()
# print(optimize)
# print(optimize._strategy)

[*********************100%***********************]  1 of 1 completed


Start                     2000-01-03 00:00:00
End                       2022-11-25 00:00:00
Duration                   8362 days 00:00:00
Exposure Time [%]                        4.09
Equity Final [$]                  10417667.49
Equity Peak [$]                   10672888.26
Return [%]                               4.18
Buy & Hold Return [%]                   36.30
Return (Ann.) [%]                        0.17
Volatility (Ann.) [%]                    1.75
Sharpe Ratio                             0.10
Sortino Ratio                            0.15
Calmar Ratio                             0.03
Max. Drawdown [%]                       -6.71
Avg. Drawdown [%]                       -1.70
Max. Drawdown Duration     3069 days 00:00:00
Avg. Drawdown Duration      670 days 00:00:00
# Trades                                   17
Win Rate [%]                            35.29
Best Trade [%]                           4.65
Worst Trade [%]                         -2.71
Avg. Trade [%]                    